# Práctica: Calidad del aire de Madrid <span style="color:#21610B">LUIS BLANCO</span>

El objetivo de la práctica es afianzar y demostrar los conocimientos adquiridos en el modelado y gestión de datos en Cassandra.

Para ello habra que realizar los siguientes ejercicios:
* Pensar un caso de uso acorde al conjunto de datos proporcionado.
* Crear el modelo de datos acorde al caso de uso que se ha pensado.
* Crear las tablas necesarias en Cassandra.
* Tratar e insertar los datos en las tablas creadas.
* Realizar las consultas necesarias para el caso de uso pensado.

Qué hay que entregar:
* Descripción del caso de uso y justificación de por qué es adecuado realizarlo con Cassandra.
* Diagrama del modelo de datos.
* Un notebook de Jupyter que contenga:
    * * Todas las sentencias para crear las tablas
    * * El código necesario para tratar los datos e insertarlos en Cassandra.
    * * Las sentencias de consulta
* Conclusiones:
    * * Qué te parece Cassandra como data store.
    * * Qué te ha parecido el ejercicio.
    * * Qué has aprendido.
    * * Qué has hechado de menos.
    * * Cómo mejorarías la práctica.

### <p style="color:#21610B;background-color:#E1F5A9;padding:20px">**0. Librerías:**

In [3]:
import pandas as pd
import numpy as np
import glob
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

## El conjunto de datos

El conjunto de datos hace referencia a la calidad del aire de la ciudad de Madrid.
Es el conjunto de mediciones horarias de distintas métricas sobre la calidad del aire de Madrid durante el año 2017.

De dónde se han sacado los datos: 
* Portal de datos abiertos del Ayuntamiento de Madrid
* https://datos.madrid.es/sites/v/index.jsp?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD
* https://datos.madrid.es/sites/v/index.jsp?vgnextoid=9e42c176313eb410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD

Qué contienen los ficheros:
* 12 ficheros con las mediciones horarias, uno por cada mes.
    * * Formato CSV
* 1 Fichero con la información de las estaciones de medición.
    * * Formato CSV
* 1 Fichero con la descripción de los datos y el procedimiento de medición.
    * * Formato PDF
    
### Ficheros con mediciones

Listado de campos:
* PROVINCIA = Código numérico de la provincia (28).
* MUNICIPIO = Código numérico del municipio (079).
* ESTACION = Número de estación.
* MAGNITUD = Magnitud que se está midiendo. Ver Anexo II "Magnitudes, unidades y técnicas de medida".
* PUNTO_MUESTREO = Tiene el siguiente formato XXYYYZZZ_M_TT. XXYYYZZZ representa la estación de medida.
    * * XX = Códido de provincia
    * * YYY = Código de municipio
    * * ZZZ = Número de estación
    * * M = Magnitud
    * * TT = Técnica de medida
* ANO = Año en el que se tomó la medición.
* MES = Mes del año en el que se tomó la medición
* DIA = Día del mes en el que se tomó la medición.
* H01 - H24 = Hora del día en el que se tomó la medición.
* V01 - V24 = Indica si la medición es válida. ÚNICAMENTE SON VÁLIDOS LOS DATOS QUE LLEVAN EL CÓDIGO "V"

In [9]:
df_data = pd.read_csv("data/2017/abr_mo17.csv", header = 0, sep = ";")
print(df_data.info())
df_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4490 entries, 0 to 4489
Data columns (total 56 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PROVINCIA       4490 non-null   int64  
 1   MUNICIPIO       4490 non-null   int64  
 2   ESTACION        4490 non-null   int64  
 3   MAGNITUD        4490 non-null   int64  
 4   PUNTO_MUESTREO  4490 non-null   object 
 5   ANO             4490 non-null   int64  
 6   MES             4490 non-null   int64  
 7   DIA             4490 non-null   int64  
 8   H01             4490 non-null   float64
 9   V01             4490 non-null   object 
 10  H02             4490 non-null   float64
 11  V02             4490 non-null   object 
 12  H03             4490 non-null   float64
 13  V03             4490 non-null   object 
 14  H04             4490 non-null   float64
 15  V04             4490 non-null   object 
 16  H05             4490 non-null   float64
 17  V05             4490 non-null   o

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2017,4,1,2.0,V,2.0,V,2.0,V,1.0,V,2.0,V,2.0,V,3.0,V,3.0,V,3.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V
1,28,79,4,1,28079004_1_38,2017,4,2,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,7.0,V,6.0,V,5.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,28,79,4,1,28079004_1_38,2017,4,3,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
3,28,79,4,1,28079004_1_38,2017,4,4,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2017,4,5,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,6.0,V,8.0,V,6.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V
5,28,79,4,1,28079004_1_38,2017,4,6,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,8.0,V,9.0,V,7.0,V,4.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,3.0,V,2.0,V,2.0,V
6,28,79,4,1,28079004_1_38,2017,4,7,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
7,28,79,4,1,28079004_1_38,2017,4,8,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
8,28,79,4,1,28079004_1_38,2017,4,9,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,2.0,N,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V
9,28,79,4,1,28079004_1_38,2017,4,10,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,1.0,V,1.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V


### Fichero de estaciones

Listado de campos:
* NUMERO = Código de estación.
* ESTACION = Nombre de la estación. 
* DIRECCION = Dirección de la estación.
* LONGITUD = Logitud geográfica de la ubicación de la estación. En grados, minutos y segundos.
* LATITUD = Latitud geográfica de la ubicación de la estación. En grados, minutos y segundos.
* ALTITUD = Altitud sobre el nivel del mar en metros.
* TIPO ESTACION:
    * * UT = Urbana de tráfico
    * * UF = Urbana de fondo
    * * S = Suburbanam
* CONTAMINANTE MEDIDO = Magnitud que se está midiendo. Ver Anexo II "Magnitudes, unidades y técnicas de medida".
    * * NO2;SO2;CO;PM10;PM2,5;O3;BTX;HC = Sólo mide las magnitudes que esténa 1.
* SENSORES METEOROLÓGICOS
    * * UV;VV;DV;TMP;HR;PRB;RS;LL = Sólo mide las magnitudes que esténa 1.


In [8]:
df_estaciones = pd.read_csv("data/estaciones.csv", header = 0, sep = ';')
print(df_estaciones.info())
df_estaciones.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NUMERO          24 non-null     int64  
 1   ESTACION        24 non-null     object 
 2   DIRECCION       24 non-null     object 
 3   LONGITUD        24 non-null     object 
 4   LATITUD         24 non-null     object 
 5   ALTITUD         24 non-null     int64  
 6   TIPO ESTACION   24 non-null     object 
 7   NO2             24 non-null     int64  
 8   SO2             10 non-null     float64
 9   CO              9 non-null      float64
 10  PM10            12 non-null     float64
 11  PM2,5           6 non-null      float64
 12  O3              14 non-null     float64
 13  BTX             6 non-null      float64
 14  HC              3 non-null      float64
 15  UV              4 non-null      float64
 16  VV              5 non-null      float64
 17  DV              5 non-null      float

,NUMERO,ESTACION,DIRECCION,LONGITUD,LATITUD,ALTITUD,TIPO ESTACION,NO2,SO2,CO,PM10,"PM2,5",O3,BTX,HC,UV,VV,DV,TMP,HR,PRB,RS,LL
0,4,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN
1,8,Escuelas Aguirre,Entre C/ Alcala y C/ O’ Donell,"3º 40' 56,35''O","40º 25' 17,63''N",670,UT,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,Avda. Ramon y Cajal,Avda. Ramon y Cajal esq. C/ Príncipe de Vergara,"3º 40' 38,48''O","40º 27' 05,31''N",708,UT,1,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,16,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,"3º 38' 21,24''O","40º 26' 24,17''N",693,UF,1,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,17,Villaverde,C/. Juan Penalver,"3º 42' 47,96''O","40º 20' 49,70''N",604,UF,1,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<p style="background-color:#D8D8D8;padding:1px">

# <p style="color:#21610B;text-align:center">**MADRID SMOG FREE TOWER PROJECT**
    
<img src="./Images/smogfreetower.jpg" title="Madrid Smog Free Tower Project">

### <p style="color:#21610B;background-color:#E1F5A9;padding:20px">**1. Descripción del caso de uso y justificación de por qué es adecuado realizarlo con Cassandra:**

<span style="color:#21610B">**Descripción del caso de uso:**
    
<span style="color:#21610B">Para tratar de reducir la contaminación atmosférica en la ciudad de Madrid, se va a llevar a ejecución un proyecto pionero en el mundo, el **"Madrid Smog Free Tower Project"**. Este proyecto consiste en la instalación de 3 purificadores reducidos, capaces de aspirar hasta 10.000 metros cúbicos de aire contaminado (NO2) a la hora.
    
<span style="color:#21610B">**+Info: https://www.studioroosegaarde.net/project/smog-free-tower**
    
<span style="color:#21610B">Para la realización de este ejercicio, vamos a suponer que las torres pueden ser móviles. La torres se reubicarán cada hora en los puntos de mayor contaminación, por lo que será necesario conocer los puntos de mayor contaminación para desplazar las torres a esos puntos.

<span style="color:#21610B">La práctica estará dividida en dos partes, una inicial, para conocer los puntos de mayor contaminación anuales, y así saber dónde colocar las torres inicialmente; y una segunda parte con los datos más actuales (2020), que nos indiquen los puntos de mayor contaminación cada hora y así colocar las torres en aquellos lugares.
    
<span style="color:#21610B">**¿Por qué es adecuado realizarlo con Cassandra?**

- <span style="color:#21610B">Es necesaria una alta capacidad de escritura, ya que se recogen datos de mediciones de los contaminantes cada hora.  
    
- <span style="color:#21610B">Tendremos lecturas rápidas gracias a que se realizarán mediante primary-key. Se generaran tantas tablas como queries necesitemos, esto implica que no es necesario realizar joins para unificar tablas.  
    
- <span style="color:#21610B">La arquitectura de Cassandra es tolerante a fallos.  
    
- <span style="color:#21610B">La base de datos de Cassandra se distribuye entre diferentes nodos, y gracias a su particionado mediante primary-key podremos conseguir uniformidad entre los diferentes nodos.  
    
- <span style="color:#21610B">Este proyecto, pionero en España, es facilmente replicable en otras ciudades, ya que Cassandra es facilmente escalable y al no estar centralizada, podrían crearse datacenters en cada ciudad.

### <p style="color:Green;background-color:#E1F5A9;padding:20px">**2. Diagrama del modelo de datos:**

<span style="color:#21610B">**1ª Parte:** Número de torres necesarias: **((ano),contaminante_valor,estacion,validez))**

<img align=left src="./Images/numero_de_torres.jpg" title="KEYSPACE: madrid_smog_free_tower_project
TABLE: Número de torres">

<span style="color:#21610B">Las queries serán:

- <span style="color:#21610B">Las 10 concentraciones más altas en 2017 viendo la estación en la que fue.
- <span style="color:#21610B">Las 10 concentraciones más altas en 2018 viendo la estación en la que fue.
- <span style="color:#21610B">Las 10 concentraciones más altas en 2019 viendo la estación en la que fue.
- <span style="color:#21610B">Las 10 concentraciones más altas en 2020 viendo la estación en la que fue.

<span style="color:#21610B">**2ª Parte:** Reubicación de torres a la hora: **((fecha, hora), valor_contaminante)**

<img align=left src="./Images/reubicacion_torres.jpg" title="KEYSPACE: madrid_smog_free_tower_project
TABLE: Reubicación de las torres">

<span style="color:#21610B">Las queries serán:

- <span style="color:#21610B">Los 3 puntos con una mayor concentración a una hora dada.

### <p style="color:Green;background-color:#E1F5A9;padding:20px">**3. Sentencias para creación de tablas, tratamiento e insercción de los datos en Cassandra y sentencias de consulta:**
    
<span style="color:#21610B">**Creación cluster de Cassandra:**

In [4]:
auth_provider = PlainTextAuthProvider(
    username='cassandra', password='cassandra')
cluster = Cluster(contact_points=['127.0.0.1'], port=9042, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.execute("DROP KEYSPACE IF EXISTS madrid_smog_free_tower_project")
session.execute("CREATE KEYSPACE madrid_smog_free_tower_project WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1}")
session.set_keyspace("madrid_smog_free_tower_project")

<span style="color:#21610B">**Carga de datos de mediciones de los años 2017, 2018, 2019 y 2020:**

<span style="color:#21610B">Para la realización de la práctica, vamos a utilizar, además de los datos de 2017, los datos de los años 2018, 2019 y 2020 (enero y febrero), que hemos descargado en la misma web que los aportados para la práctica.  
Por lo que vamos a unir los datos de estos años a nuestra tabla anterior, antes de subirlos a nuestro cluster.  

<p style="color:#21610B;text-align:right">*Hemos unificado todos los años, desde 2017 a 2020, en la carpeta adjunta DataMad.

In [4]:
archivos=glob.glob("./data/DataMad/*.csv")

listado= []
for j in archivos:
    df=pd.read_csv(j, sep=";", index_col=None, header=0)
    listado.append(df)
    
Cont_Mad= pd.concat(listado,axis=0,ignore_index=True)

In [5]:
pd.set_option('display.max_columns', 500)
print(Cont_Mad["ANO"].unique())
Cont_Mad.head(10)

[2017 2018 2019 2020]


,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2017,4,1,2.0,V,2.0,V,2.0,V,1.0,V,2.0,V,2.0,V,3.0,V,3.0,V,3.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V
1,28,79,4,1,28079004_1_38,2017,4,2,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,7.0,V,6.0,V,5.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,28,79,4,1,28079004_1_38,2017,4,3,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
3,28,79,4,1,28079004_1_38,2017,4,4,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2017,4,5,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,6.0,V,8.0,V,6.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V
5,28,79,4,1,28079004_1_38,2017,4,6,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,8.0,V,9.0,V,7.0,V,4.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,3.0,V,2.0,V,2.0,V
6,28,79,4,1,28079004_1_38,2017,4,7,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
7,28,79,4,1,28079004_1_38,2017,4,8,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
8,28,79,4,1,28079004_1_38,2017,4,9,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,2.0,N,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V
9,28,79,4,1,28079004_1_38,2017,4,10,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,1.0,V,1.0,V,2.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V,2.0,V,2.0,V,1.0,V,1.0,V,1.0,V


<span style="color:#21610B">**Tratamiento de los datos:**
    
<span style="color:#21610B">Renombramos la columna ESTACION de Cont_Mad por NUMERO, para poder nuestros dataframes tengan el mismo nombre y así poder unirlas:

In [6]:
Cont_Mad=Cont_Mad.rename(columns={"ESTACION": "NUMERO"})
Cont_Mad.shape

(178360, 56)

In [10]:
Cont_Mad_union = Cont_Mad.merge(df_estaciones, how="inner", on=["NUMERO"])
Cont_Mad_union.head()

,PROVINCIA,MUNICIPIO,NUMERO,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24,ESTACION,DIRECCION,LONGITUD,LATITUD,ALTITUD,TIPO ESTACION,NO2,SO2,CO,PM10,"PM2,5",O3,BTX,HC,UV,VV,DV,TMP,HR,PRB,RS,LL
0,28,79,4,1,28079004_1_38,2017,4,1,2.0,V,2.0,V,2.0,V,1.0,V,2.0,V,2.0,V,3.0,V,3.0,V,3.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN
1,28,79,4,1,28079004_1_38,2017,4,2,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,7.0,V,6.0,V,5.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN
2,28,79,4,1,28079004_1_38,2017,4,3,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN
3,28,79,4,1,28079004_1_38,2017,4,4,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN
4,28,79,4,1,28079004_1_38,2017,4,5,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,6.0,V,8.0,V,6.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN


In [11]:
Cont_Mad_union.shape

(178360, 78)

<span style="color:#21610B">Vamos a ver los valores que toma la columna MAGNITUD y a que contaminantes corresponden con los datos del Anexo II del archivo "Interprete_ficheros_ calidad_ del_ aire_global". Sólo nos quedaremos con el NO2, que es lo que consigue limpiar nuestras torres. Comprobamos que la magnitud del NO2 es el 8:

In [12]:
mag=Cont_Mad_union["MAGNITUD"].unique()

In [13]:
cont=["SO2","CO","NO","NO2","NOX","PM2.5","PM10","O3","TOL","BEN","EBE","TCH","CH4","NMHC"]

In [14]:
mag_cont= pd.DataFrame({"MAGNITUD":mag, "CONTAMINANTE":cont})
mag_cont

,MAGNITUD,CONTAMINANTE
0,1,SO2
1,6,CO
2,7,NO
3,8,NO2
4,12,NOX
5,9,PM2.5
6,10,PM10
7,14,O3
8,20,TOL
9,30,BEN


In [15]:
Cont_Mad_def= Cont_Mad_union.merge(mag_cont, how="inner", on=["MAGNITUD"])
Cont_Mad_def.head()

,PROVINCIA,MUNICIPIO,NUMERO,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24,ESTACION,DIRECCION,LONGITUD,LATITUD,ALTITUD,TIPO ESTACION,NO2,SO2,CO,PM10,"PM2,5",O3,BTX,HC,UV,VV,DV,TMP,HR,PRB,RS,LL,CONTAMINANTE
0,28,79,4,1,28079004_1_38,2017,4,1,2.0,V,2.0,V,2.0,V,1.0,V,2.0,V,2.0,V,3.0,V,3.0,V,3.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,SO2
1,28,79,4,1,28079004_1_38,2017,4,2,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,7.0,V,6.0,V,5.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,SO2
2,28,79,4,1,28079004_1_38,2017,4,3,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,SO2
3,28,79,4,1,28079004_1_38,2017,4,4,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,SO2
4,28,79,4,1,28079004_1_38,2017,4,5,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,3.0,V,6.0,V,8.0,V,6.0,V,4.0,V,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,SO2


In [16]:
Cont_Mad_def= Cont_Mad_def.loc[Cont_Mad_def["MAGNITUD"]==8]
Cont_Mad_def

,PROVINCIA,MUNICIPIO,NUMERO,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24,ESTACION,DIRECCION,LONGITUD,LATITUD,ALTITUD,TIPO ESTACION,NO2,SO2,CO,PM10,"PM2,5",O3,BTX,HC,UV,VV,DV,TMP,HR,PRB,RS,LL,CONTAMINANTE
51917,28,79,4,8,28079004_8_8,2017,4,1,21.0,V,19.0,V,17.0,V,24.0,V,17.0,V,35.0,V,40.0,V,36.0,V,34.0,V,35.0,V,44.0,V,44.0,V,38.0,V,26.0,V,29.0,V,22.0,V,17.0,V,24.0,V,32.0,V,46.0,V,57.0,V,82.0,V,85.0,V,77.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2
51918,28,79,4,8,28079004_8_8,2017,4,2,67.0,V,45.0,V,37.0,V,21.0,V,19.0,V,25.0,V,50.0,V,86.0,V,92.0,V,85.0,V,68.0,V,54.0,V,46.0,V,37.0,V,30.0,V,21.0,V,19.0,V,21.0,V,20.0,V,22.0,V,26.0,V,38.0,V,15.0,V,15.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2
51919,28,79,4,8,28079004_8_8,2017,4,3,14.0,V,9.0,V,8.0,V,4.0,V,4.0,V,15.0,V,15.0,V,37.0,V,55.0,V,58.0,V,46.0,V,32.0,V,30.0,V,25.0,V,22.0,V,17.0,V,17.0,V,22.0,V,24.0,V,23.0,V,27.0,V,30.0,V,23.0,V,20.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2
51920,28,79,4,8,28079004_8_8,2017,4,4,8.0,V,3.0,V,2.0,V,3.0,V,2.0,V,4.0,V,15.0,V,39.0,V,59.0,V,49.0,V,39.0,V,36.0,V,36.0,V,25.0,V,19.0,V,14.0,V,15.0,V,16.0,V,15.0,V,14.0,V,19.0,V,22.0,V,28.0,V,23.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2
51921,28,79,4,8,28079004_8_8,2017,4,5,20.0,V,23.0,V,19.0,V,17.0,V,18.0,V,28.0,V,46.0,V,59.0,V,63.0,V,74.0,V,80.0,V,74.0,V,64.0,V,48.0,V,39.0,V,33.0,V,28.0,V,34.0,V,33.0,V,45.0,V,68.0,V,99.0,V,93.0,V,84.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80267,28,79,60,8,28079060_8_8,2019,9,26,20.0,V,15.0,V,17.0,V,17.0,V,13.0,V,16.0,V,21.0,V,39.0,V,39.0,V,32.0,V,27.0,V,21.0,V,22.0,V,26.0,V,25.0,V,27.0,V,20.0,V,17.0,V,19.0,V,23.0,V,38.0,V,66.0,V,55.0,V,46.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2
80268,28,79,60,8,28079060_8_8,2019,9,27,36.0,V,44.0,V,38.0,V,25.0,V,27.0,V,24.0,V,31.0,V,28.0,V,26.0,V,16.0,V,24.0,V,24.0,V,24.0,V,30.0,V,23.0,V,17.0,V,14.0,V,15.0,V,17.0,V,18.0,V,38.0,V,53.0,V,60.0,V,68.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2
80269,28,79,60,8,28079060_8_8,2019,9,28,52.0,V,41.0,V,62.0,V,29.0,V,13.0,V,20.0,V,19.0,V,17.0,V,23.0,V,23.0,V,17.0,V,17.0,V,16.0,V,15.0,V,16.0,V,16.0,V,14.0,V,13.0,V,12.0,V,17.0,V,34.0,V,57.0,V,55.0,V,46.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2
80270,28,79,60,8,28079060_8_8,2019,9,29,37.0,V,45.0,V,40.0,V,21.0,V,19.0,V,28.0,V,24.0,V,23.0,V,20.0,V,15.0,V,14.0,V,15.0,V,18.0,V,18.0,V,15.0,V,10.0,V,9.0,V,10.0,V,12.0,V,20.0,V,29.0,V,32.0,V,43.0,V,38.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2


<span style="color:#21610B">Vamos a crear una columna de fecha en su formato string:

In [17]:
Cont_Mad_def["FECHA"] = pd.to_datetime(Cont_Mad_def["ANO"].astype(str) 
                                       + "/" + Cont_Mad_def["MES"].astype(str)
                                       + "/" +Cont_Mad_def["DIA"].astype(str))

In [18]:
Cont_Mad_def["FECHA"]

51917   2017-04-01
51918   2017-04-02
51919   2017-04-03
51920   2017-04-04
51921   2017-04-05
           ...    
80267   2019-09-26
80268   2019-09-27
80269   2019-09-28
80270   2019-09-29
80271   2019-09-30
Name: FECHA, Length: 28355, dtype: datetime64[ns]

In [18]:
Cont_Mad_def[["FECHA"]] = Cont_Mad_def[["FECHA"]].astype(str)
Cont_Mad_def.dtypes

PROVINCIA           int64
MUNICIPIO           int64
NUMERO              int64
MAGNITUD            int64
PUNTO_MUESTREO     object
ANO                 int64
MES                 int64
DIA                 int64
H01               float64
V01                object
H02               float64
V02                object
H03               float64
V03                object
H04               float64
V04                object
H05               float64
V05                object
H06               float64
V06                object
H07               float64
V07                object
H08               float64
V08                object
H09               float64
V09                object
H10               float64
V10                object
H11               float64
V11                object
                   ...   
H22               float64
V22                object
H23               float64
V23                object
H24               float64
V24                object
ESTACION           object
DIRECCION   

In [19]:
Cont_Mad_def

,PROVINCIA,MUNICIPIO,NUMERO,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24,ESTACION,DIRECCION,LONGITUD,LATITUD,ALTITUD,TIPO ESTACION,NO2,SO2,CO,PM10,"PM2,5",O3,BTX,HC,UV,VV,DV,TMP,HR,PRB,RS,LL,CONTAMINANTE,FECHA
51917,28,79,4,8,28079004_8_8,2017,4,1,21.0,V,19.0,V,17.0,V,24.0,V,17.0,V,35.0,V,40.0,V,36.0,V,34.0,V,35.0,V,44.0,V,44.0,V,38.0,V,26.0,V,29.0,V,22.0,V,17.0,V,24.0,V,32.0,V,46.0,V,57.0,V,82.0,V,85.0,V,77.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2,2017-04-01
51918,28,79,4,8,28079004_8_8,2017,4,2,67.0,V,45.0,V,37.0,V,21.0,V,19.0,V,25.0,V,50.0,V,86.0,V,92.0,V,85.0,V,68.0,V,54.0,V,46.0,V,37.0,V,30.0,V,21.0,V,19.0,V,21.0,V,20.0,V,22.0,V,26.0,V,38.0,V,15.0,V,15.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2,2017-04-02
51919,28,79,4,8,28079004_8_8,2017,4,3,14.0,V,9.0,V,8.0,V,4.0,V,4.0,V,15.0,V,15.0,V,37.0,V,55.0,V,58.0,V,46.0,V,32.0,V,30.0,V,25.0,V,22.0,V,17.0,V,17.0,V,22.0,V,24.0,V,23.0,V,27.0,V,30.0,V,23.0,V,20.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2,2017-04-03
51920,28,79,4,8,28079004_8_8,2017,4,4,8.0,V,3.0,V,2.0,V,3.0,V,2.0,V,4.0,V,15.0,V,39.0,V,59.0,V,49.0,V,39.0,V,36.0,V,36.0,V,25.0,V,19.0,V,14.0,V,15.0,V,16.0,V,15.0,V,14.0,V,19.0,V,22.0,V,28.0,V,23.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2,2017-04-04
51921,28,79,4,8,28079004_8_8,2017,4,5,20.0,V,23.0,V,19.0,V,17.0,V,18.0,V,28.0,V,46.0,V,59.0,V,63.0,V,74.0,V,80.0,V,74.0,V,64.0,V,48.0,V,39.0,V,33.0,V,28.0,V,34.0,V,33.0,V,45.0,V,68.0,V,99.0,V,93.0,V,84.0,V,Pza. de Espana,Plaza de Espana,"3º 42' 44,09''O","40º 25' 25,87''N",635,UT,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NO2,2017-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80267,28,79,60,8,28079060_8_8,2019,9,26,20.0,V,15.0,V,17.0,V,17.0,V,13.0,V,16.0,V,21.0,V,39.0,V,39.0,V,32.0,V,27.0,V,21.0,V,22.0,V,26.0,V,25.0,V,27.0,V,20.0,V,17.0,V,19.0,V,23.0,V,38.0,V,66.0,V,55.0,V,46.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2,2019-09-26
80268,28,79,60,8,28079060_8_8,2019,9,27,36.0,V,44.0,V,38.0,V,25.0,V,27.0,V,24.0,V,31.0,V,28.0,V,26.0,V,16.0,V,24.0,V,24.0,V,24.0,V,30.0,V,23.0,V,17.0,V,14.0,V,15.0,V,17.0,V,18.0,V,38.0,V,53.0,V,60.0,V,68.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2,2019-09-27
80269,28,79,60,8,28079060_8_8,2019,9,28,52.0,V,41.0,V,62.0,V,29.0,V,13.0,V,20.0,V,19.0,V,17.0,V,23.0,V,23.0,V,17.0,V,17.0,V,16.0,V,15.0,V,16.0,V,16.0,V,14.0,V,13.0,V,12.0,V,17.0,V,34.0,V,57.0,V,55.0,V,46.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2,2019-09-28
80270,28,79,60,8,28079060_8_8,2019,9,29,37.0,V,45.0,V,40.0,V,21.0,V,19.0,V,28.0,V,24.0,V,23.0,V,20.0,V,15.0,V,14.0,V,15.0,V,18.0,V,18.0,V,15.0,V,10.0,V,9.0,V,10.0,V,12.0,V,20.0,V,29.0,V,32.0,V,43.0,V,38.0,V,Tres Olivos,Plaza Tres Olivos,"3º 41' 23,14""O","40º 30' 02,12''N",715,UF,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NO2,2019-09-29


<span style="color:#21610B">**Creación de tablas:**
    
<span style="color:#21610B">1ª Parte - numero_de_torres:

In [20]:
session.execute(
    """
    CREATE TABLE numero_de_torres (
    ano int,
    contaminante_valor float,
    estacion text,
    validez text,
    PRIMARY KEY ((ano),contaminante_valor,estacion,validez))
    WITH CLUSTERING ORDER BY (contaminante_valor DESC)
    """
)

<span style="color:#21610B">Vamos a crear dos listas, una con las columnas con datos (Hs) y otra con su validez (Vs), para así poder insertarlo en nuestras tablas:

In [21]:
lista_horas=[]
for horas in list(range(1,25)):
    hora_string = str(horas)
    if len(hora_string)<2:
        hora_string = "0" + hora_string
    hora_string = "H" + hora_string
    lista_horas.append(hora_string)
lista_horas

['H01',
 'H02',
 'H03',
 'H04',
 'H05',
 'H06',
 'H07',
 'H08',
 'H09',
 'H10',
 'H11',
 'H12',
 'H13',
 'H14',
 'H15',
 'H16',
 'H17',
 'H18',
 'H19',
 'H20',
 'H21',
 'H22',
 'H23',
 'H24']

In [22]:
lista_valor=[]
for valor in list(range(1,25)):
    valor_string=str(valor)
    if len(valor_string)<2:
        valor_string="0"+valor_string
    valor_string="V"+ valor_string
    lista_valor.append(valor_string)
lista_valor

['V01',
 'V02',
 'V03',
 'V04',
 'V05',
 'V06',
 'V07',
 'V08',
 'V09',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24']

In [23]:
for index, torres in Cont_Mad_def.head(100).iterrows():
#for index, torres in Cont_Mad_def.iterrows():
    """Sólo ingreso los 100 primeros datos porque el tiempo de carga es muy largo."""
    for horas in lista_horas:
        for valores in lista_valor:
            session.execute(
                """
                INSERT INTO numero_de_torres (ano, contaminante_valor,estacion,validez)
                VALUES (%s, %s, %s,%s)
                """,
                (torres["ANO"], torres[horas],torres["ESTACION"],torres[valores])
            )

In [24]:
numero_de_torres = session.execute(
    """
    SELECT * FROM numero_de_torres 
    WHERE ANO=2017 LIMIT 10
    """
)
for row in numero_de_torres:
    print(row)

Row(ano=2017, contaminante_valor=114.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=108.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=105.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=103.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=102.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=101.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=97.0, estacion='Pza. de Espana', validez='N')
Row(ano=2017, contaminante_valor=97.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=93.0, estacion='Pza. de Espana', validez='V')
Row(ano=2017, contaminante_valor=90.0, estacion='Pza. de Espana', validez='V')


In [25]:
numero_de_torres = session.execute(
    """
    SELECT * FROM numero_de_torres 
    WHERE ANO=2018 LIMIT 10
    """
)
for row in numero_de_torres:
    print(row)

Row(ano=2018, contaminante_valor=152.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=135.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=131.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=127.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=126.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=121.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=120.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=118.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=116.0, estacion='Pza. de Espana', validez='V')
Row(ano=2018, contaminante_valor=113.0, estacion='Pza. de Espana', validez='V')


In [26]:
numero_de_torres = session.execute(
    """
    SELECT * FROM numero_de_torres 
    WHERE ANO=2019 LIMIT 10
    """
)
for row in numero_de_torres:
    print(row)

Row(ano=2019, contaminante_valor=95.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=94.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=93.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=92.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=88.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=85.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=84.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=83.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=81.0, estacion='Pza. de Espana', validez='V')
Row(ano=2019, contaminante_valor=80.0, estacion='Pza. de Espana', validez='V')


In [27]:
numero_de_torres = session.execute(
    """
    SELECT * FROM numero_de_torres 
    WHERE ANO=20 LIMIT 10
    """
)
for row in numero_de_torres:
    print(row)

<span style="color:#21610B">Vamos a colocar las torres teniendo en cuenta cada año cuál ha sido la estación que más concentración ha tenido(cada año), al tener que coger parte del dataset por el tiempo, no han dado resultados para el año 2020.  
Vamos a colocar las torres en las 3 estaciones que más contaminan, en este caso, al no tener todos los datos, sólo nos aparece Plaza de España, con todos los datos, elegiríamos las tres estaciones donde más concentración tengamos.

<span style="color:#21610B">reubicacion_torres:

In [28]:
session.execute(
    """
    CREATE TABLE reubicacion_torres (
    fecha date,
    hora int,
    contaminante_valor float,
    direccion text,
    estacion text,
    validez text,
    PRIMARY KEY ((fecha,hora),contaminante_valor,direccion,estacion,validez))
    WITH CLUSTERING ORDER BY (contaminante_valor DESC)
    """
)

In [29]:
for index, ubicacion in Cont_Mad_def.head(100).iterrows():
#for index, ubicacion in Cont_Mad_def.iterrows():
    """Sólo ingreso los 100 primeros datos porque el tiempo de carga es muy largo."""
    for horas in lista_horas:
        for valores in lista_valor:
            session.execute(
                """
                INSERT INTO reubicacion_torres (fecha,hora,contaminante_valor,direccion,estacion,validez)
                VALUES (%s, %s, %s,%s,%s,%s)
                """,
                (ubicacion["FECHA"], int(horas[1:]), ubicacion[horas],ubicacion["DIRECCION"],
                 ubicacion["ESTACION"],ubicacion[valores])
            )

In [31]:
reubicacion_torres = session.execute(
    """
    SELECT * FROM reubicacion_torres
    WHERE fecha='2018-01-10' AND hora=7 LIMIT 3
    """
)
for row in reubicacion_torres:
    print(row)

Row(fecha=Date(17541), hora=7, contaminante_valor=40.0, direccion='Plaza de Espana', estacion='Pza. de Espana', validez='V')


<span style="color:#21610B">Al no coger todos los datos, hemos cogido una fecha y hora que diera resultados, pero no hemos podido encontrar una que nos ofrezca los tres resultados que le pedimos, en este caso sería la estación Pza.de Espana, en la dirección Plaza de Espana.  
    Con todos los datos cargados, obtendríamos los tres puntos con mayor concentración de NO2 y allí trasladaríamos nuestras torres. Si uno de ellos siguiera siendo el punto en el que está, esa torre permanecearía estática otra hora.

### <p style="color:Green;background-color:#E1F5A9;padding:20px">**4. Conclusiones:**
    
<span style="color:#21610B">**Qué te parece Cassandra como data store:**
    
<span style="color:#21610B">El principal problema es la necesidad de crear una tabla para cada query, al ser query-driven. Además, el tiempo de insertación de datos en las tablas es muy lento en un PC normal (he tenido que incluir menos datos que los de nuestro dataset por el tiempo que tardaba), aunque una vez insertados los datos, la respuesta a las queries es muy rápida. Además, es distribuido, con lo que puede almacenar gran cantidad de datos.

<span style="color:#21610B">**Qué te ha parecido el ejercicio:**
    
<span style="color:#21610B">El ser una práctica abierta y con datos reales, incita a investigar, aumentando el aprendizaje.

<span style="color:#21610B">**Qué has aprendido:**
    
<span style="color:#21610B">He aprendido a usar Cassandra, es decir, crear tablas, insertar datos y a pensar siempre teniendo en cuenta primero qué es lo que quiero obtener. Además, he tenido que mejorar mis conocimientos en Python, Pandas y HTML.

<span style="color:#21610B">**Qué has hechado de menos:**

<span style="color:#21610B">Poder poner en práctica Kafka, que aunque es posible, dependiendo del modelo de uso que haga cada alumno será innecesario (como es mi caso).

<span style="color:#21610B">**Cómo mejorarías la práctica:**
    
<span style="color:#21610B">Como he dicho antes, la ventaja de un caso de uso personal es bueno, pero también dificulta el uso de Kafka, además, en este sentido, una práctica algo más guiada para no necesitar tanto en tiempo en "buscarte la vida" sería mejor, ya que así podríamos ver usos más realistas como en la vida real.